In [ ]:
import pathlib
import datetime
import json
from collections import defaultdict
import pathlib

from readability import Readability
import seaborn
import matplotlib.pyplot as plt
import pandas as pd
import ollama

CWD = pathlib.Path("").parent

In [ ]:

prompt_res_table = defaultdict(list)
for assignment_name, df in zip(assignment_names, all_dfs):
    text = texts[assignment_name]
    summary = summaries[assignment_name]
    prompt = generate_prompt(assignment_name, df, text, summary)

    res = ollama.chat(
        model="llama3",
        messages=[
            {
                "role": "user",
                "content": json.dumps(prompt)
            }
        ]
    )
    
    content = res['message']['content']
    # Write into a markdown file
    with open(f"./data/reports/{assignment_name}.md", "w") as f:
        f.write(content)

    # Store the results in a table
    prompt_res_table['assignment'].append(assignment_name)
    prompt_res_table['prompt'].append(prompt)
    prompt_res_table['response'].append(content)

    break

# Save the results into a table
timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
prompt_res_df = pd.DataFrame(prompt_res_table)
prompt_res_df.to_excel(CWD / 'data' / 'prompt_responses' / f"res_{timestamp}.xlsx", index=False)